# Notebook 1: Data Clean-Up and Pre-Selection

In [207]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import glob
import pandas as pd

# World Bank Development Data

Reading in and scouting the data (available from https://databank.worldbank.org/source/world-development-indicators) for the year 2019.

Since we do not plan to analyse our data over time, we focus on the year 2019.
We choose the year 2019 over the year 2020, because we are not interested in the effect of COVID-19 on happiness but more general correlations. Furthermore, due to COVID-19, the 2020 World Happiness Report (WHR) has a lower sample size. 

In [208]:
wb_data = pd.read_csv("data/world_bank_data.csv")
wb_data.head()

#wb_data.tail()

,Country Name,Country Code,Series Name,Series Code,2019 [YR2019]
0,Australia,AUS,Access to electricity (% of population),EG.ELC.ACCS.ZS,100
1,Australia,AUS,Adjusted savings: carbon dioxide damage (% of ...,NY.ADJ.DCO2.GN.ZS,1.11294140526482
2,Australia,AUS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,10.9366
3,Australia,AUS,Adolescents out of school (% of lower secondar...,SE.SEC.UNER.LO.ZS,1.97146999835968
4,Australia,AUS,"Adolescents out of school, female (% of female...",SE.SEC.UNER.LO.FE.ZS,2.19272994995117


Dropping the last five rows, because they contain only NaN values and Meta-Data, as well as the Series Code and Country Code column, because they are not needed for our analysis.

In [209]:
print(wb_data.shape)

wb_data = wb_data.dropna(subset = ["Series Name"])
wb_data = wb_data.drop(columns=["Series Code","Country Code"])

print(wb_data.shape)

(383843, 5)
(383838, 3)


Reformatting the data such that the development indicators, e.g. *Access to electricity (% of population)*, each get their own column.

In [210]:
# combining the development indicators (Series Name) into columns by country
wb_data = wb_data.set_index(["Country Name", "Series Name"], drop=True).unstack("Series Name")

# removing the upper level of the multi-index 
wb_data.columns = wb_data.columns.droplevel(0)

# adding a columns with country names for later reformatting purposes
wb_data["Country Name"] = wb_data.index

### World Happiness Report 2019

Reading in the World Happiness Report (WHR) Data (available from https://www.kaggle.com/unsdsn/world-happiness).

In [211]:
whr_data = pd.read_csv("data/whr_2020.csv")
whr_data.shape

(153, 20)

Indentifying countries for which we have data from both the World Bank (WB) and World Happiness Report 2019 (WHR).

In [212]:
whr_countries = list(whr_data["Country name"])
wb_countries = list(wb_data["Country Name"])

# countries that are listed in both
intersect_countries = set(wb_countries).intersection(whr_countries)

We suspect that some countries are excluded due to different naming conventions. Hence, we look at the countries that are not in the intersection, but present in the WHR since it contains fewer countries and no global regions (e.g. Western Europe). 

In [213]:
# countries that are either not in the WB or the WHR data
xor_countries = set(wb_countries)^set(whr_countries)

# countries that are only in the WHR data
ex_whr_countries = set(whr_countries).intersection(xor_countries)

print(xor_countries, ex_whr_countries)
print(len(xor_countries), len(ex_whr_countries))

{'Iran, Islamic Rep.', 'Sub-Saharan Africa (IDA & IBRD countries)', 'Macedonia', 'Hong Kong S.A.R. of China', 'Channel Islands', 'Northern Mariana Islands', 'Congo (Brazzaville)', 'Kyrgyzstan', 'Sint Maarten (Dutch part)', 'Russian Federation', 'Venezuela, RB', 'Not classified', 'Kyrgyz Republic', 'Middle income', "Cote d'Ivoire", 'Middle East & North Africa (IDA & IBRD countries)', 'Guyana', 'French Polynesia', 'St. Martin (French part)', 'Djibouti', 'West Bank and Gaza', 'St. Lucia', 'Dominica', 'Papua New Guinea', 'Middle East & North Africa', 'Liechtenstein', 'Samoa', 'Eritrea', 'Slovak Republic', 'Grenada', 'Seychelles', 'Syrian Arab Republic', 'Europe & Central Asia (IDA & IBRD countries)', 'Timor-Leste', 'East Asia & Pacific (excluding high income)', 'Gibraltar', 'Bhutan', 'Small states', 'Turks and Caicos Islands', 'Angola', 'South Asia (IDA & IBRD)', 'Low & middle income', 'Marshall Islands', 'Micronesia, Fed. Sts.', 'Monaco', 'Oman', 'OECD members', 'North Macedonia', 'Lao PD

We compare this list to the WB data base by hand and find that Taiwan and Cyprus are not present.

We also identify 19 country name pairs that refer to the same country. This is due to different naming conventions, official renaming and typos. 

In [214]:
# list of tuples (whr name, wb name)
happy_country_list_renamed = [
("Swaziland", "Eswatini"),
("Macedonia", "North Macedonia"),
("Kyrgyzstan", "Kyrgyz Republic"),
("Yemen", "Yemen, Rep."),
("Gambia", "Gambia, The"),
("Ivory Coast", "Cote d'Ivoire"),
("Venezuela", "Venezuela, RB"),
("South Korea", "Korea, Rep."),
("Laos", "Lao PDR"),
("Hong Kong S.A.R. of China", "Hong Kong SAR, China"),
("Congo (Kinshasa)", "Congo, Dem. Rep."),
("Palestinian Territories", "West Bank and Gaza"),
("Congo (Brazzaville)", "Congo, Rep."),
("Iran", "Iran, Islamic Rep."),
("Egypt", "Egypt, Arab Rep."),
("Russia", "Russian Federation"),
("Slovakia", "Slovak Republic"),
("Syria", "Syrian Arab Republic")]

Since we want to unify the data, we first rename the WHR data. We chose the WB naming conventions, because they are more up-to-date (Macedonia was renamed North Macedonia in 2019). Then we remove Taiwan and Cyprus from the WHR data and use the WHR country list to select the remaining countries in the WB data (i.e. remove the rest).

In [215]:
# renaming the countries in the whr data
for happy_name, wb_name in happy_country_list_renamed:
    for i in range(0, len(whr_data["Country name"])):
        if whr_data.loc[i, "Country name"] == happy_name:
            whr_data.loc[i, "Country name"] = wb_name
            
# removing Taiwan and Cyprus from the whr_data
whr_data.index = whr_data["Country name"]
whr_data = whr_data.drop(["Taiwan Province of China", "North Cyprus"])

wb_data = wb_data[wb_data["Country Name"].isin(whr_data["Country name"])]
wb_data.index = wb_data["Country Name"]
print(wb_data.shape, whr_data.shape)

(151, 1444) (151, 20)


Additionally, since we are working with incomplete data, we want to indentify the development indicators with the most available data, so we can exclude especially sparse ones. Therefore, we set a sparsity threshold for the number of countries a specific indicator is allowed to lack data for. Indicators that are missing data for more countries than the specified thresholds are removed.

In [216]:
sparsity_threshold = 1 #0, 5, 10
delete_columns = []

def delete_sparse_countries(df, sparsity_thresh):
    """
    a function that deletes indicators from the data, if they are missing data 
    from more countries than specified by the sparsity_threshold.
    
    returns: the dense data
    """
    for i in range(0, len(df.columns)):
        num_values = sum(df[df.columns[i]]=="..")
        if num_values > sparsity_thresh:
            delete_columns.append(df.columns[i])
            
    dense_df = df.drop(columns=delete_columns)
    
    return dense_df
        
#wb_data = delete_sparse_countries(wb_data, sparsity_thresh)
#print(len(wb_data_dense.columns))

We find that dropping every indicator with more than 1 missing country data point yields 180 countries.
Dropping every indicator with more than 0 missing data points on the other hand shrinks the country set to 30.


Subsequently, we find that a 100% dense data set - while retaining 156 indicators - is possible given one condition: <br>
We have to delete Kosovo.

In [217]:
from collections import Counter

def count_sparse_countries(df, sparsity_thresh):
    """
    a function that iterates through the indicators and counts a country 
    if it is the only one without data for a given indicator.
    
    returns: the occurences
    """
    sparse_countries = []
    for i in range(0, len(df.columns)):
        num_values = sum(df[df.columns[i]]=="..")
        if num_values == sparsity_thresh:
            sparse_countries.append(list(df[df[df.columns[i]]==".."].index.values))
            
    # merges the list of lists into single list        
    sparse_countries_unified = sum(sparse_countries, [])
    
    #counts and ouputs occurances of countries
    print(Counter(sparse_countries_unified))
            
count_sparse_countries(wb_data, 1)

# delete Kosovo from dataset
wb_data.drop("Kosovo", inplace=True)
whr_data.drop("Kosovo", inplace=True)

# delete all indicators with missing data points
wb_data = delete_sparse_countries(wb_data, 0)
print(len(wb_data.columns))

Counter({'Kosovo': 126, 'Turkmenistan': 24, 'Lao PDR': 2, 'Venezuela, RB': 1, 'Libya': 1, 'Panama': 1, 'Hong Kong SAR, China': 1})
157


Lastly, we save the curated data sets to csv-files. 

In [218]:
sorted(wb_data.columns)

['Access to electricity (% of population)',
 'Access to electricity, urban (% of urban population)',
 'Adjusted savings: energy depletion (current US$)',
 'Adjusted savings: mineral depletion (current US$)',
 'Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Age dependency ratio (% of working-age population)',
 'Age dependency ratio, old (% of working-age population)',
 'Age dependency ratio, young (% of working-age population)',
 'Birth rate, crude (per 1,000 people)',
 'Contributing family workers, female (% of female employment) (modeled ILO estimate)',
 'Contributing family workers, male (% of male employment) (modeled ILO estimate)',
 'Contributing family workers, total (% of total employment) (modeled ILO estimate)',
 'Country Name',
 'Death rate, crude (per 1,000 people)',
 'Employers, female (% of female employment) (modeled ILO estimate)',
 'Employers, male (% of male employment) (modeled ILO estimate)',
 'Employers, total (% of total employment) (modeled ILO 

In [219]:
redundant_indicators = [
    'Age dependency ratio (% of working-age population)',
    'Contributing family workers, total (% of total employment) (modeled ILO estimate)',
    'Employers, total (% of total employment) (modeled ILO estimate)',
    'Employment in agriculture (% of total employment) (modeled ILO estimate)',
    'Employment in industry (% of total employment) (modeled ILO estimate)',
    'Employment in services (% of total employment) (modeled ILO estimate)',
    'Employment to population ratio, 15+, total (%) (modeled ILO estimate)',
    'Employment to population ratio, ages 15-24, total (%) (modeled ILO estimate)',
    'Labor force participation rate for ages 15-24, total (%) (modeled ILO estimate)',
    'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)',
    'Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate)',
    'Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)',
    'Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)',
    'Life expectancy at birth, total (years)',
    'Population ages 0-14 (% of total population)',
    'Population ages 0-14, female',
    'Population ages 0-14, male',
    'Population ages 0-14, total',
    'Population ages 15-64 (% of total population)',
    'Population ages 15-64, female',
    'Population ages 15-64, male',
    'Population ages 15-64, total',
    'Population ages 65 and above (% of total population)',
    'Population ages 65 and above, female',
    'Population ages 65 and above, male',
    'Population ages 65 and above, total',
    'Population, female',
    'Population, male',
    'Rural population',
    'Secure Internet servers',
    'Self-employed, total (% of total employment) (modeled ILO estimate)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)',
    'Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)',
    'Urban population',
    'Vulnerable employment, total (% of total employment) (modeled ILO estimate)',
    'Wage and salaried workers, total (% of total employment) (modeled ILO estimate)'
]
len(redundant_indicators)

36

In [220]:
wb_data_short = wb_data.drop(labels=redundant_indicators, axis=1)
print("shape of wb_data: ", wb_data.shape)
print("shape of wb_data_short: ", wb_data_short.shape)

shape of wb_data:  (150, 157)
shape of wb_data_short:  (150, 121)


In [221]:
wb_data.to_csv("data/wb_data.csv")
wb_data_short.to_csv("data/wb_data_short.csv")
whr_data.to_csv("data/whr_data.csv")